In [ ]:
import sys
import os

import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import matplotlib.animation as animation

sys.path.insert(0, os.path.abspath("../cli"))
from simulation.sonar import Sonar
from simulation.utils import CircleBottom, run_beam, positions_line
from simulation.plotting import plot_velocity

from examples.seismic import WaveletSource, Receiver

plt.rcParams['figure.figsize'] = (5, 5)

In [ ]:
class GaborSource(WaveletSource):
    """
    A source object that encapsulates everything necessary for injecting a
    Gabor source into the computational domain.

    Returns:
        The source term that will be injected into the computational domain.
    """

    def __init_finalize__(self, *args, **kwargs):
        super(GaborSource, self).__init_finalize__(*args, **kwargs)

    @property
    def wavelet(self):
        assert self.f0 is not None
        agauss = 0.5 * self.f0
        tcut = self.t0 or 1.5 / agauss
        s = (self.time_values - 20 * tcut) * agauss
        a = self.a or 1
        return a * np.exp(-0.5 * s**2) * np.cos(2 * np.pi * s)

    @property
    def signal_packet(self):
        assert self.f0 is not None
        return self.wavelet[: np.searchsorted(self.time_values, 4 * 2 / self.f0)]

In [ ]:
v_env = 1.5
ns = 128
domain_size = (10, 10)
source_distance = 0.002
cy = cy = (ns - 1) / 2 * source_distance + source_distance
f0 = 100